# 4.1 - Procesos asíncronos


![async](images/async.png)




**[Documentación](https://docs.python.org/3/library/asyncio.html)**


**asyncio** es una biblioteca para escribir código [concurrente](https://es.wikipedia.org/wiki/Concurrencia_(inform%C3%A1tica)) utilizando la sintaxis async/await. Se utiliza como base en múltiples frameworks asíncronos de Python y provee un alto rendimiento en redes y servidores web, bibliotecas de conexión de base de datos, colas de tareas distribuidas, etc.

Suele encajar perfectamente para operaciones con límite de E/S y código de red estructurado de alto nivel. Además provee un conjunto de APIs de alto nivel para:

+ ejecutar corutinas de Python de manera concurrente y tener control total sobre su ejecución

+ realizar redes E/S y comunicación entre procesos(IPC)

+ controlar subprocesos

+ distribuir tareas a través de colas

+ sincronizar código concurrente

Adicionalmente, existen APIs de bajo nivel para desarrolladores de bibliotecas y frameworks para:

+ crear y administrar bucles de eventos, los cuales proveen APIs asíncronas para redes, ejecutando subprocesos, gestionando señales del sistema operativo, etc..

+ implementar protocolos eficientes utilizando transportes

+ bibliotecas puente basadas en retrollamadas y código con sintaxis async/wait

$$$$

Nosotros nos enfocaremos en el uso de bucles de eventos para la extracción de datos de la web.


### Hola Mundo

In [ ]:
import asyncio


# funcion asincrona
async def saludar():
    
    print('Holaaaa..')
    
    await asyncio.sleep(3)
    
    print('pero que te pashaaa')
    
    
await saludar()            # en jupyter
#asyncio.run(saludar())    # en un .py

### Ejemplo response

Comprobando la respuesta de tres urls.

In [ ]:
import requests as req

In [ ]:
url='https://s3-eu-west-1.amazonaws.com/'

req.get(url)

In [ ]:
urls=[
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/breakfast.jpg',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/forbidden',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/the-html5-breakfast-site.html'
]

In [ ]:
for e in urls:
    print(req.get(e))

In [ ]:
req.get(e).status_code

In [ ]:
# de manera asincrona

async def comprobar():
    
    bucle = asyncio.get_event_loop()   # bucle asincrono
    
    futuros=[]
    
    for e in urls:
        
        promesa=bucle.run_in_executor(None, req.get, e)
        
        futuros.append(promesa)
        
    for res in await asyncio.gather(*futuros): # aqui se los pido, dame los response
        print(res.status_code)
        
        
await comprobar()

### Ejemplo ESPN

Volvamos al ejemplo de scrapeo de la págine de ESPN. Vamos a realizar múltiples requests para obtener los datos de todos los equipos.


https://www.espn.com/soccer/competitions

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import time

import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager

PATH=ChromeDriverManager().install()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
url='https://www.espn.com/soccer/competitions'

In [ ]:
driver=webdriver.Chrome(PATH)

driver.get(url)

# cookies
aceptar=driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
aceptar.click()

time.sleep(4)

In [ ]:
# selecciona equipos laliga
equipos=driver.find_element(By.XPATH, '//*[@id="fittPageContainer"]/div[3]/div/div[1]/div/div[2]/div[2]/div/div[4]/div/section/div/div/span[2]/a')
equipos.click()

time.sleep(2)

In [ ]:
stats = driver.find_elements(By.CSS_SELECTOR, 'a.AnchorLink')

stats[16].get_attribute('href')

In [ ]:
stats[16]  # esto es e en el siguiente bucle

In [ ]:
team_stats = []

for e in stats:
    
    try:
        link = e.get_attribute('href')

        if 'soccer/team/stats' in link:
            team_stats.append(link)
        else:
            continue
    except:
        continue
        
driver.quit()

team_stats

In [ ]:
len(team_stats)

**Extracción asincrónica**

In [ ]:
def asincrono(funcion):
    
    def eventos(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, funcion, *args, **kwargs)
    
    return eventos

In [ ]:
DATOS=[]

CABECERAS=[]

In [ ]:
@asincrono
def extraer(url):
    
    global DATOS, CABECERAS

    # inicia el driver
    driver=webdriver.Chrome(PATH)
    driver.get(url)

    time.sleep(2)

    # acepta cookies
    aceptar=driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    aceptar.click()

    time.sleep(2)

    # disciplina
    dis=driver.find_element(By.XPATH,'//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/section/div/div[2]/nav/ul/li[2]/a')
    dis.click()

    time.sleep(2)

    tabla=driver.find_element(By.TAG_NAME,'tbody')

    filas=tabla.find_elements(By.TAG_NAME, 'tr')


    data=[]

    for f in filas:

        elementos=f.find_elements(By.TAG_NAME, 'td') 

        tmp=[]

        for e in elementos:

            tmp.append(e.text)

        tmp.append(url.split('/')[-1])
        data.append(tmp)


    cabeceras=driver.find_element(By.TAG_NAME, 'thead')

    cabeceras=[c.text for c in cabeceras.find_elements(By.TAG_NAME, 'th')]+['TEAM']


    DATOS+=data

    CABECERAS=cabeceras

In [ ]:
%%time 

for url in team_stats[:5]:
    
    res=extraer(url)
    display(res)

In [ ]:
df=pd.DataFrame(DATOS, columns=CABECERAS)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.TEAM.unique()